## Today's mission

- 웹 데이터 수집 및  Beautiful Soup 리뷰
  - 웹 데이터 수집 개념
  - 웹 데이터 기본 구조
  - 웹 데이터 HTML 
  - Python 파이썬 관련 라이브러리 : Beautiful Soup
  
- 실습 (1) google news 검색

- 실습 (2) google news 검색 함수 작성

### (1) google news 검색
 - 참고사항 : robots.txt 
   https://searchadvisor.naver.com/guide/seo-basic-robots

- 구글 뉴스(http://news.google.com/) 

- URL 인코딩으로 한글 >> 유니코드 변환 https://www.convertstring.com/ko/EncodeDecode/UrlEncode

In [1]:
# 패키지 import
import requests
from bs4 import BeautifulSoup

#### google news 검색 참고:

https://news.google.com/search?q=%EC%BD%94%EB%A1%9C%EB%82%98&hl=ko&gl=KR&ceid=KR%3Ako

 -> "https://news.google.com" + "/search?q=%EC%BD%94%EB%A1%9C%EB%82%98&hl=ko&gl=KR&ceid=KR%3Ako"
 - (검색어: 코로나)

In [2]:
# HTML 파일 읽어들이기
base_url = "https://news.google.com"

In [3]:
search_url = "/search?q=%EC%BD%94%EB%A1%9C%EB%82%98&hl=ko&gl=KR&ceid=KR%3Ako"
url = base_url + search_url
print(url)

https://news.google.com/search?q=%EC%BD%94%EB%A1%9C%EB%82%98&hl=ko&gl=KR&ceid=KR%3Ako


In [4]:
# 검색정보 url GET 방식으로 요청
response = requests.get(url)
response                     # 200 : 서버 응답 정상
                             # 404 : url이 잘못되거나 오류

<Response [200]>

In [5]:
# HTML parser 이용해 soup 객체 생성
soup = BeautifulSoup(response.content, 'html.parser') # response.text도 확인
# soup

In [6]:
# 내부 HTML 전체 구조파악 :
## Type1 : soup.find_all('div', class_='xrnccd')[0]
## Type2 : soup.select('div.xrnccd')[0]

In [7]:
news_items = soup.select('div.xrnccd')
print(len(news_items))
# print(news_items[0])

100


In [8]:
## 내부HTML 각 내용 구조파악 :
## link
news_items[1].find('a', attrs={'class':'VDXfz'})

<a aria-hidden="true" class="VDXfz" href="./articles/CBMiOmh0dHBzOi8vbmV3cy5zYnMuY28ua3IvbmV3cy9lbmRQYWdlLmRvP25ld3NfaWQ9TjEwMDcwMjA0OTnSATdodHRwczovL25ld3Muc2JzLmNvLmtyL2FtcC9uZXdzLmFtcD9uZXdzX2lkPU4xMDA3MDIwNDk5?hl=ko&amp;gl=KR&amp;ceid=KR%3Ako" jslog="95014; 5:W251bGwsbnVsbCxudWxsLG51bGwsbnVsbCxudWxsLG51bGwsbnVsbCwiaHR0cHM6Ly9uZXdzLnNicy5jby5rci9uZXdzL2VuZFBhZ2UuZG8_bmV3c19pZFx1MDAzZE4xMDA3MDIwNDk5Il0=; track:click,vis" jsname="hXwDdf" tabindex="-1" target="_blank"></a>

In [9]:
news_items[1].find('a', attrs={'class':'VDXfz'}).get('href')

'./articles/CBMiOmh0dHBzOi8vbmV3cy5zYnMuY28ua3IvbmV3cy9lbmRQYWdlLmRvP25ld3NfaWQ9TjEwMDcwMjA0OTnSATdodHRwczovL25ld3Muc2JzLmNvLmtyL2FtcC9uZXdzLmFtcD9uZXdzX2lkPU4xMDA3MDIwNDk5?hl=ko&gl=KR&ceid=KR%3Ako'

##### news_items 에서 각 뉴스의 링크, 제목, 내용, 출처, 등록일시 등 추출
- 오류 났을시 직접 사이트에 들어가 F12를 누른 후 찾아보기

In [10]:
## title (제목)
news_items[0].find('a', attrs={'class':'DY5T1d'}).text

'중국 ‘제로 코로나’의 종말…국경 열고 ‘B급 감염병’ 강등[딥다이브]'

In [11]:
## content (내용)
news_items[0].find('h3', attrs={'class':'ipQwMb ekueJc RD0gLb'}).text

'중국 ‘제로 코로나’의 종말…국경 열고 ‘B급 감염병’ 강등[딥다이브]'

In [12]:
## register (등록사 : ex. 언론사 이름)
news_items[0].find('a', attrs={'class':'wEwyrc AVN2gc WfKKme'}).text

'동아일보'

In [13]:
## time
news_items[0].find('time', attrs={'class':'WW6dff uQIVzc Sksgp slhocf'}).text

'9시간 전'

In [14]:
## datetime
news_items[0].find('time', attrs={'class':'WW6dff uQIVzc Sksgp slhocf'}).text
news_items[0].find('time', attrs={'class':'WW6dff uQIVzc Sksgp slhocf'}).get('datetime')
news_items[0].find('time', attrs={'class':'WW6dff uQIVzc Sksgp slhocf'}).get('datetime').split('T')

['2022-12-26', '23:01:00Z']

### (2) google news 검색 함수 작성

 - (1)을 활용하여 검색어를 직접 입력하여 구글 뉴스 스크래핑 작성

In [15]:
# 패키지 import
import requests
from bs4 import BeautifulSoup
import urllib

In [16]:
# 찾아볼 뉴스 관련단어 입력
# 검색어 요청 :
input_word = input('검색어 입력 : ')

검색어 입력 : IT


In [17]:
## Google news 검색어 URL 코드로 encoding
urllib.parse.quote(input_word) # URL로 변환시켜줌

'IT'

In [18]:
# 검색어 정보 URL 구성
base_url = "https://news.google.com"
keyword = urllib.parse.quote(input_word)
search_url = "/search?q=" + keyword + "&hl=ko&gl=KR&ceid=KR%3Ako"  # keyword: %EC%BD%94%EB%A1%9C%EB%82%98
url = base_url + search_url
print(url)

https://news.google.com/search?q=IT&hl=ko&gl=KR&ceid=KR%3Ako


##### 검색 함수 생성

In [19]:
def google_news_clipping_keyword(keyword_input):
    base_url = "https://news.google.com"
    keyword = urllib.parse.quote(keyword_input)
    search_url = "/search?q=" + keyword + "&hl=ko&gl=KR&ceid=KR%3Ako"  # keyword: %EC%BD%94%EB%A1%9C%EB%82%98
    url = base_url + search_url
    
    return url

In [20]:
# 함수를 실행하여 뉴스 목록 정리
# 찾아볼 뉴스 관련단어 입력
search_word = input("검색어를 입력하세요: ")

검색어를 입력하세요: IT


In [21]:
news = google_news_clipping_keyword(search_word)

print(news)

https://news.google.com/search?q=IT&hl=ko&gl=KR&ceid=KR%3Ako
